##### Raw Alpha Vantage Api (https://www.alphavantage.co/documentation/)

In [1]:
import requests
import json
from io import StringIO
import pandas as pd

def save_quarterly_data(symbol, returning=False):
    alpha_key = ''
    with open('keys.json', 'r') as keys:
        credentials = json.load(keys)
        alpha_key = credentials.get('alphavantage')
    
    if not alpha_key: return('You need an api key for Alpha Vantage')
    
    values = get_data(symbol, alpha_key)
    if values is None:
        print(f'{symbol} not saved !!')
        return None
    
    income = requests.get(f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&outputsize=full&apikey={alpha_key}')
    if 'quarterlyReports' not in json.loads(income.text):
        print(f'{symbol} not saved !!')
        return None
    income = pd.read_json(StringIO(json.dumps(json.loads(income.text)['quarterlyReports'])))
    
    if 'fiscalDateEnding' not in income.columns:
        print(f'{symbol} not saved !!')
        return None
    
    balance = requests.get(f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&outputsize=full&apikey={alpha_key}')
    if 'quarterlyReports' not in json.loads(balance.text):
        print(f'{symbol} not saved !!')
        return None
    balance = pd.read_json(StringIO(json.dumps(json.loads(balance.text)['quarterlyReports'])))
    
    data = pd.merge(income, balance, on='fiscalDateEnding')
    
    cashflow = requests.get(f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&outputsize=full&apikey={alpha_key}')
    if 'quarterlyReports' not in json.loads(cashflow.text):
        print(f'{symbol} not saved !!')
        return None
    cashflow = pd.read_json(StringIO(json.dumps(json.loads(cashflow.text)['quarterlyReports'])))
    
    data = pd.merge(data, cashflow, on='fiscalDateEnding')
    
    data = last_clossing(values, data)
    data = append_y(data)
    
    try: data.to_csv(f'./datasets/fundamental_data/{symbol}.csv')
    except: print(f'{symbol} not saved !!')
    
    if returning: return data

In [2]:
def get_data(symbol, alpha_key):
    csv = requests.get(f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={alpha_key}&datatype=csv')
    data = pd.read_csv(StringIO(csv.text))
    
    if 'timestamp' not in data.columns or 'close' not in data.columns:
        return None
    
    return data[['timestamp', 'close']]

In [3]:
def last_clossing(values, data):
    for f in data['fiscalDateEnding']:
        g = f
        clossingVal = 'nan'
        while clossingVal == 'nan':
            try: clossingVal = values.loc[values['timestamp'] == g, 'close'].values[0]
            except: g = g[:-2]+str(int(g[-2:])-1)
                
            if int(f[-2:])-int(g[-2:]) > 2: break
        
        data.loc[data['fiscalDateEnding'] == f, 'clossingVal'] = clossingVal
        
    return data

In [4]:
def append_y(data):
    future = None
    for f in data['fiscalDateEnding']:
        if future is not None: data.loc[data['fiscalDateEnding'] == f, 'nextClossingVal'] = future
        future = data.loc[data['fiscalDateEnding'] == f, 'clossingVal'].values[0]
        
    return data

In [5]:
symbols = pd.read_csv('./datasets/nasdaq_list.csv', sep='|')
symbols = symbols[['Symbol']].to_numpy().reshape((len(symbols))).tolist()

In [ ]:
# Me voy descargando todos los datos posibles a un ritmo adecuado dentro de lo que permite la API
# (5 peticiones por minuto, 500 por día)

import time

for symbol in symbols[432:]:
    save_quarterly_data(symbol)
    time.sleep(30)

BND not saved !!
BNDW not saved !!
BNDX not saved !!
BNGO not saved !!
BNR not saved !!
BNTC not saved !!
BOKF not saved !!
BOMN not saved !!
BOTJ not saved !!
BOTZ not saved !!
BOWXU not saved !!
BPMC not saved !!
BPOPN not saved !!
BPTH not saved !!
BPYPO not saved !!
BPYU not saved !!
BREW not saved !!
BRKR not saved !!
BRLIR not saved !!
BRLIW not saved !!
BROGW not saved !!
BRPA not saved !!
BRPAU not saved !!
BSAE not saved !!
BSBE not saved !!
BSCE not saved !!
BSCK not saved !!
BSCL not saved !!
BSCM not saved !!
BSCN not saved !!
BSCO not saved !!
BSCP not saved !!
BSCQ not saved !!
BSCR not saved !!
BSCS not saved !!
BSCT not saved !!
BSDE not saved !!
BSGM not saved !!
BSJK not saved !!
BSJL not saved !!
BSJM not saved !!
BSJN not saved !!
BSJO not saved !!
BSJP not saved !!
BSJQ not saved !!
BSJR not saved !!
BSML not saved !!
BSMM not saved !!
BSMN not saved !!
BSMO not saved !!
BSMP not saved !!
BSMQ not saved !!
BSMR not saved !!
BSMS not saved !!
BSMT not saved !!
BSTC 